In [68]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
import os
import csv
import time
import datetime
import shutil
from jinja2 import Environment, FileSystemLoader

In [57]:
# Open the browser
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": f"{os.getcwd()}/downloads"
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

/var/folders/hq/ypf9892n1s94fwgm0p6k6k8m0000gn/T/ipykernel_35540/3579103318.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
/var/folders/hq/ypf9892n1s94fwgm0p6k6k8m0000gn/T/ipykernel_35540/3579103318.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [58]:
# Get username and password from .env file
load_dotenv()
USER = os.getenv("USERNAME")
PASS = os.getenv("PASS")
COURSE_NO = os.getenv("COURSE_NO")
ASSIGNMENT_NO = os.getenv("ASSIGNMENT_NO")
BASE_URL = "https://www.gradescope.com"

In [59]:
# Log in
login_url = f"{BASE_URL}/login"
driver.get(login_url)
username_input = driver.find_element("id", "session_email")
password_input = driver.find_element("id", "session_password")
submit = driver.find_element("name", "commit")
username_input.send_keys(USER)
password_input.send_keys(PASS)
submit.click()

In [67]:
# Go to assignment "review grades" page
download_url = f"{BASE_URL}/courses/{COURSE_NO}/assignments/{ASSIGNMENT_NO}/scores.csv"
driver.get(download_url)
try:
    os.remove(f"{os.getcwd()}/downloads/csv/scores.csv")
except FileNotFoundError:
    pass
while True:
    time.sleep(1)
    files = os.listdir("downloads/")
    found = False
    for file in files:
        if ".csv" in file:
            os.rename(f"downloads/{file}", f"downloads/csv/scores.csv")
            found = True
    if found:
        break

In [61]:
# Read CSV to get URLS, names, and TFs
tfs = {}
with open("last_sweep.txt", "r") as f:
    last_sweep = f.read()

with open("downloads/csv/scores.csv", "r") as f:
    reader = csv.DictReader(f)
    most_recent = last_sweep
    for row in reader:
        tf = row["Sections"]
        if tf not in tfs:
            tfs[tf] = []
        if row["Submission Time"]:
            if row["Submission Time"] > most_recent:
                most_recent = row["Submission Time"]
            tfs[tf].append({
                "student": row["Name"],
                "link": f"{BASE_URL}/courses/{COURSE_NO}/assignments/{ASSIGNMENT_NO}/submissions/{row    ['Submission ID']}.zip",
                "submission_id": row["Submission ID"],
                "sub_time": row["Submission Time"]
            })

with open("last_sweep.txt", "w") as f:
    f.write(most_recent)

In [62]:
# Setup TF directories:
for tf in tfs:
    if not os.path.isdir(f"downloads/{tf}"):
        os.makedirs(f"downloads/{tf}")

In [66]:
def manage_one_sub(tf, student, link, sub_id):
    # Download submission
    driver.get(link)
    while True:
        time.sleep(1)
        if f"{sub_id}.zip" in os.listdir("downloads/"):
            break
    os.rename(f"downloads/{sub_id}.zip", f"downloads/{tf}/{student}.zip")
    shutil.unpack_archive(f"downloads/{tf}/{student}.zip", f"downloads/{tf}/{student}")

for tf, students in tfs.items():
    for student in students:
        if student["sub_time"] > last_sweep:
            print(f"Downloading Submission {student['submission_id']}")
            manage_one_sub(tf, student["student"], student["link"], student["submission_id"])

Kamryn Ohly


In [76]:
# Create index.html
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("template.html")
for tf, students in tfs.items():
    students = {s["student"]: f"./{s['student']}/index.html" for s in students}
    content = template.render(
        tf=tf,
        students=students
    )
    with open(f"downloads/{tf}/index.html", "w") as f:
        f.write(content)
